In [28]:
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


import cv2
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import os
import matplotlib.image as img
import scipy.ndimage


In [29]:
print(torch.__version__)

1.8.1+cu101


In [30]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [5]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 19.5MB/s 
     |████████████████████████████████| 952kB 35.8MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=1f81f90a1f8cca782419ca7d30b167de680705ebec40877ea9b8a0d695f35792
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [31]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

In [5]:
train_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=300),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.CenterCrop(height=128, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
        Resize(height=299, width=299),
        ToTensorV2(),
    ]
)


val_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=300),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
        Resize(height=299, width=299),
        ToTensorV2(),
    ]
)


In [33]:
image_path='/content/drive/MyDrive/emergency_vs_non-emergency_dataset/images/'
df=pd.read_csv('/content/drive/MyDrive/emergency_vs_non-emergency_dataset/emergency_train.csv')

In [37]:
df.head()

,image,label
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [36]:
df=df.rename(columns={'image_names':'image','emergency_or_not':'label'})

In [38]:
df['label'].value_counts()

0    965
1    681
Name: label, dtype: int64

In [39]:
df.isnull().sum()

image    0
label    0
dtype: int64

In [40]:
train, valid = train_test_split(df, stratify=df.label, test_size=0.2)

In [41]:
import imageio
image_list=[]
width_list=[]
height_list=[]
channel_list=[]
image_list=df['image'].values
for item in range(len(image_list)):
    current_item=image_list[item]
    current_image=image_path+str(current_item)
    height, width, channels = imageio.imread(current_image).shape
    width_list.append(width)
    height_list.append(height)
    channel_list.append(channels)

In [42]:
for i in channel_list:
    if i == 3:
        print('yes')
    else:
        print('No')      

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [43]:
avg_channel=sum(channel_list)/len(channel_list)
avg_channel

3.0

In [44]:
class Datasetloader(Dataset):
    def __init__(self, data, path , transform = None):
        super().__init__()
        self.data = data.values
        self.path = path
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        img_name,label = self.data[index]
        img_path = os.path.join(self.path, img_name)
        image = img.imread(img_path)
        imag = self.transform(image=image)["image"]
        return imag, label

In [45]:
train_data = Datasetloader(train, image_path, train_transform)
valid_data =Datasetloader(valid, image_path, val_transform)


In [46]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [47]:
train_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset = valid_data, batch_size = 8, shuffle=False, num_workers=0)

In [48]:
model = models.googlenet(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [21]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images.shape)
print(labels.shape)

torch.Size([8, 3, 299, 299])
torch.Size([8])


In [49]:
num_epochs = 35
num_classes = 2
batch_size = 25
learning_rate = 0.001

In [50]:
train_losses = []
valid_losses = []

for epoch in range(1, num_epochs + 1):
   
    train_loss = 0.0
    valid_loss = 0.0
    
  
    model.train()
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
 
    model.eval()
    for data, target in valid_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item() * data.size(0)
    
  
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
        
   
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

Epoch: 1 	Training Loss: 0.539071 	Validation Loss: 0.408096
Epoch: 2 	Training Loss: 0.409150 	Validation Loss: 0.374460
Epoch: 3 	Training Loss: 0.402929 	Validation Loss: 0.343885
Epoch: 4 	Training Loss: 0.384186 	Validation Loss: 0.359429
Epoch: 5 	Training Loss: 0.353747 	Validation Loss: 0.340506
Epoch: 6 	Training Loss: 0.370119 	Validation Loss: 0.310450
Epoch: 7 	Training Loss: 0.346786 	Validation Loss: 0.313705
Epoch: 8 	Training Loss: 0.351474 	Validation Loss: 0.425477
Epoch: 9 	Training Loss: 0.350248 	Validation Loss: 0.435654
Epoch: 10 	Training Loss: 0.382950 	Validation Loss: 0.307560
Epoch: 11 	Training Loss: 0.339632 	Validation Loss: 0.315550
Epoch: 12 	Training Loss: 0.322457 	Validation Loss: 0.308117
Epoch: 13 	Training Loss: 0.341800 	Validation Loss: 0.390404
Epoch: 14 	Training Loss: 0.352547 	Validation Loss: 0.331020
Epoch: 15 	Training Loss: 0.352613 	Validation Loss: 0.313262
Epoch: 16 	Training Loss: 0.338933 	Validation Loss: 0.302462
Epoch: 17 	Traini

In [55]:
correct_count, all_count = 0, 0
for data, target in valid_loader:
     data =  data.to(device)
     target = target.to(device)
     with torch.no_grad():
        logps = model(data)
     ps = torch.exp(logps)
     probab = list(ps.cpu()[0])
     pred_label = probab.index(max(probab))
     true_label = labels.cpu()[i]
     if(true_label == pred_label):
       correct_count += 1
     all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 42

Model Accuracy = 0.6904761904761905
